# e10s vs non-e10s stats for 46.0a2

In [1]:
import IPython

from __future__ import division
from moztelemetry.spark import get_pings, get_pings_properties
import numpy as np
import datetime

%pylab inline
IPython.core.pylabtools.figsize(16, 7)

Unable to parse whitelist (/home/hadoop/anaconda2/lib/python2.7/site-packages/moztelemetry/histogram-whitelists.json). Assuming all histograms are acceptable.
Populating the interactive namespace from numpy and matplotlib


In [2]:
PING_OPTIONS = {
    "app": "Firefox",
    "channel": "aurora",
    "version": "46.0a2",
    "build_id": "*", 
    "submission_date": ("20160125", "20160328"),
    "fraction": 1,
}
main_pings = get_pings(sc, doc_type="main", **PING_OPTIONS)
main_ping_data = get_pings_properties(main_pings,
                                      ["meta/submissionDate",
                                       "environment/settings/e10sEnabled",
                                       "payload/info/subsessionLength",
                                       "payload/keyedHistograms/SUBPROCESS_CRASHES_WITH_DUMP/content",
                                       "payload/keyedHistograms/SUBPROCESS_CRASHES_WITH_DUMP/plugin"])

crash_pings = get_pings(sc, doc_type="crash", **PING_OPTIONS)
crash_ping_data = get_pings_properties(crash_pings,
                                       ["meta/submissionDate",
                                        "environment/settings/e10sEnabled"])

In [3]:
def SeqOp1(acc, d):
    if d["environment/settings/e10sEnabled"]:
        n = 3
    else:
        n = 0

    content = d["payload/keyedHistograms/SUBPROCESS_CRASHES_WITH_DUMP/content"]
    if content is not None:
        acc[n + 0] += content

    plugin = d["payload/keyedHistograms/SUBPROCESS_CRASHES_WITH_DUMP/plugin"]
    if plugin is not None:
        acc[n + 1] += plugin
    
    length = d["payload/info/subsessionLength"]
    if length is not None:
        acc[n + 2] += length
    
    return acc

def SeqOp2(acc, d):
    if d["environment/settings/e10sEnabled"]:
        acc[1] += 1
    else:
        acc[0] += 1
        
    return acc

def CombOp(acc1, acc2):
    return acc1 + acc2

def todict1(x):
    return {"e10s": {"content": x[3], "plugin": x[4], "khours": x[5] / (3600. * 1000.)},
            "non-e10s": {"content": x[0], "plugin": x[1], "khours": x[2] / (3600. * 1000.)}}

def todict2(x):
    return {"e10s": x[1],
            "non-e10s": x[0]}

In [4]:
main_agg = main_ping_data.map(lambda p: (p["meta/submissionDate"], p)).aggregateByKey(np.zeros((6,), dtype=np.int), SeqOp1, CombOp).mapValues(lambda x: todict1(x))

In [5]:
main_res = main_agg.collect()

In [6]:
crash_agg = crash_ping_data.map(lambda p: (p["meta/submissionDate"], p)).aggregateByKey(np.zeros((2,), dtype=np.int), SeqOp2, CombOp).mapValues(lambda x: todict2(x))

In [7]:
crash_res = crash_agg.collect()

In [8]:
def rate(x, y):
    return x / y if y != 0 else 0

In [9]:
def mk_date(s):
    date = datetime.datetime.strptime(s, '%Y%m%d')
    return datetime.datetime.strftime(date, '%Y-%m-%d')    

In [10]:
data = {}
for x in main_res:
    date = mk_date(x[0])
    data[date] = x[1]
    data[date]["e10s"]["browser"] = 0
    data[date]["non-e10s"]["browser"] = 0
    
for x in crash_res:
    date = mk_date(x[0])
    data[date]["e10s"]["browser"] = x[1]["e10s"]
    data[date]["non-e10s"]["browser"] = x[1]["non-e10s"]

for v in data.itervalues():
    v["e10s"]["content_rate"] = rate(v["e10s"]["content"], v["e10s"]["khours"])
    v["e10s"]["browser_rate"] = rate(v["e10s"]["browser"], v["e10s"]["khours"])
    v["e10s"]["plugin_rate"] = rate(v["e10s"]["plugin"], v["e10s"]["khours"])
    v["e10s"]["b+c_rate"] = rate((v["e10s"]["content"] + v["e10s"]["browser"]), v["e10s"]["khours"])

    v["non-e10s"]["content_rate"] = rate(v["non-e10s"]["content"], v["non-e10s"]["khours"])
    v["non-e10s"]["browser_rate"] = rate(v["non-e10s"]["browser"], v["non-e10s"]["khours"])
    v["non-e10s"]["plugin_rate"] = rate(v["non-e10s"]["plugin"], v["non-e10s"]["khours"])
    v["non-e10s"]["b+c_rate"] = rate((v["non-e10s"]["content"] + v["non-e10s"]["browser"]), v["non-e10s"]["khours"])
    
data

{'2016-01-25': {'e10s': {'b+c_rate': 154.00410677618069,
   'browser': 1,
   'browser_rate': 154.00410677618069,
   'content': 0,
   'content_rate': 0.0,
   'khours': 0.0064933333333333336,
   'plugin': 0,
   'plugin_rate': 0.0},
  'non-e10s': {'b+c_rate': 0,
   'browser': 0,
   'browser_rate': 0,
   'content': 0,
   'content_rate': 0,
   'khours': 0.0,
   'plugin': 0,
   'plugin_rate': 0}},
 '2016-01-26': {'e10s': {'b+c_rate': 50.438576610181507,
   'browser': 2,
   'browser_rate': 5.9339501894331184,
   'content': 15,
   'content_rate': 44.504626420748387,
   'khours': 0.33704361111111109,
   'plugin': 3,
   'plugin_rate': 8.9009252841496771},
  'non-e10s': {'b+c_rate': 103.39999281944493,
   'browser': 4,
   'browser_rate': 103.39999281944493,
   'content': 0,
   'content_rate': 0.0,
   'khours': 0.038684722222222225,
   'plugin': 0,
   'plugin_rate': 0.0}},
 '2016-01-27': {'e10s': {'b+c_rate': 62.887867438586582,
   'browser': 20,
   'browser_rate': 19.96440236145606,
   'content':